https://github.com/IgnacioPardo/Tecnologias_Exponenciales_2023/blob/main/Consigna_TP_Final.md

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [10]:
# Transformamos las imagenes a tensores
train = ImageDataGenerator(rescale=1.0 / 255)
tensor_train = train.flow_from_directory(
    "train", target_size=(224, 224), batch_size=32, class_mode="categorical"
)

Found 2020 images belonging to 3 classes.


In [8]:
test = ImageDataGenerator(rescale=1.0 / 255)
tensor_test = test.flow_from_directory(
    "test", target_size=(224, 224), batch_size=32, class_mode="categorical"
)

Found 120 images belonging to 3 classes.


In [9]:
validation = ImageDataGenerator(rescale=1.0 / 255)
tensor_validation = validation.flow_from_directory(
    "validation", target_size=(224, 224), batch_size=32, class_mode="categorical"
)

Found 48 images belonging to 3 classes.


In [12]:
# Hacemos el modelo
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Conv2D(
            64, (3, 3), activation="relu", input_shape=(224, 224, 3)
        ),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(3, activation="softmax"),
    ]
)

# Optimizamos el modelo como recomienda copilot chat
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Fiteamos el modelo
fitted_model = model.fit(
    tensor_train,
    steps_per_epoch=50,
    epochs=5,
    validation_data=tensor_validation,
    validation_steps=20,
)

Epoch 1/5
50/50 [==============================] - 117s 2s/step - loss: 0.4947 - accuracy: 0.8136 - val_loss: 0.2625 - val_accuracy: 0.9583
Epoch 2/5
50/50 [==============================] - 109s 2s/step - loss: 0.3715 - accuracy: 0.8690
Epoch 3/5
50/50 [==============================] - 105s 2s/step - loss: 0.2427 - accuracy: 0.9109
Epoch 4/5
50/50 [==============================] - 105s 2s/step - loss: 0.2351 - accuracy: 0.9187
Epoch 5/5
50/50 [==============================] - 106s 2s/step - loss: 0.1570 - accuracy: 0.9434


In [21]:
# Evaluamos el modelo
loss, accuracy = model.evaluate(tensor_test)

4/4 [==============================] - 2s 450ms/step - loss: 0.1889 - accuracy: 0.9500
